Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
'''
Logistic regression with L2 regularization
'''

batch_size = 128
beta = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta * tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 24.761917
Minibatch accuracy: 2.3%
Validation accuracy: 10.1%
Minibatch loss at step 500: 2.798097
Minibatch accuracy: 78.9%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 1.904527
Minibatch accuracy: 77.3%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 1.104474
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 1.085387
Minibatch accuracy: 78.9%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 0.951197
Minibatch accuracy: 78.1%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 1.036067
Minibatch accuracy: 78.9%
Validation accuracy: 81.6%
Test accuracy: 88.0%


In [7]:
'''
2 layers neural network with L2 regularization
'''
batch_size = 128
hidden_layer_size = 2048
beta = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),
        weights2)
    + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),
        weights2)
    + biases2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1107.340820
Minibatch accuracy: 8.6%
Validation accuracy: 29.6%
Minibatch loss at step 500: 387.284027
Minibatch accuracy: 85.9%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 230.387161
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 136.937714
Minibatch accuracy: 88.3%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 83.398468
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 50.460159
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 3000: 30.717833
Minibatch accuracy: 85.9%
Validation accuracy: 87.5%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
'''
only use a few batches
'''
num_steps = 3001
train_subset_size = 1280

train_dataset_subset = train_dataset[:train_subset_size]
train_labels_subset = train_labels[:train_subset_size]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_subset[offset:(offset + batch_size), :]
    batch_labels = train_labels_subset[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1152.416382
Minibatch accuracy: 10.9%
Validation accuracy: 27.3%
Minibatch loss at step 500: 380.521515
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 230.769547
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 1500: 139.951202
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 84.874184
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 2500: 51.472481
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 31.216856
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Test accuracy: 85.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
'''
2 layers neural network with dropout
'''
batch_size = 128
hidden_layer_size = 2048
beta = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  #logits = tf.nn.dropout(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2, 0.5)
  logits = tf.nn.dropout(
        tf.matmul(tf.nn.relu(
            tf.nn.dropout(tf.matmul(tf_train_dataset, weights1) + biases1, 
                0.5)), 
            weights2) + biases2, 
        0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),
        weights2)
    + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),
        weights2)
    + biases2)

In [11]:
'''
only use a few batches
'''
num_steps = 3001
train_subset_size = 1280

train_dataset_subset = train_dataset[:train_subset_size]
train_labels_subset = train_labels[:train_subset_size]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_subset[offset:(offset + batch_size), :]
    batch_labels = train_labels_subset[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1014.551453
Minibatch accuracy: 10.2%
Validation accuracy: 24.4%
Minibatch loss at step 500: 13.815273
Minibatch accuracy: 66.4%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 0.874985
Minibatch accuracy: 56.2%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 6.435724
Minibatch accuracy: 60.9%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 7.237152
Minibatch accuracy: 54.7%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 0.823402
Minibatch accuracy: 57.0%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 1.419210
Minibatch accuracy: 47.7%
Validation accuracy: 80.7%
Test accuracy: 87.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [30]:
'''
2 layers neural network with L2 regularization and dropout
'''
batch_size = 128
hidden_layer_size = 2048
beta = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.dropout(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta * tf.nn.l2_loss(weights1) + beta * tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),
        weights2)
    + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),
        weights2)
    + biases2)

In [31]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1264.001953
Minibatch accuracy: 7.0%
Validation accuracy: 11.6%
Minibatch loss at step 500: 394.752319
Minibatch accuracy: 50.8%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 229.151276
Minibatch accuracy: 56.2%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 137.288071
Minibatch accuracy: 47.7%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 83.340080
Minibatch accuracy: 49.2%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 50.821945
Minibatch accuracy: 51.6%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 31.407843
Minibatch accuracy: 46.9%
Validation accuracy: 85.0%
Minibatch loss at step 3500: 19.558842
Minibatch accuracy: 49.2%
Validation accuracy: 85.8%
Minibatch loss at step 4000: 12.284692
Minibatch accuracy: 49.2%
Validation accuracy: 86.3%
Minibatch loss at step 4500: 8.069754
Minibatch accuracy: 43.0%
Validation accuracy: 86.1%
Minibatch loss at step 5000: 5.394830
Minibatch accuracy: 48.4%
Valid

In [82]:
'''
3 layers neural network with dropouts
'''
batch_size = 128
hidden_layer1_size = 2048
hidden_layer2_size = 512

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # use smaller stddev for weights to prevent loss become NaN
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([hidden_layer1_size]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([hidden_layer2_size]))

  weights3 = tf.Variable(
    tf.truncated_normal([hidden_layer2_size, num_labels], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.dropout(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf_train_dataset, 
                        weights1
                    ) + biases1
                ), 
                weights2
            ) + biases2
        ),
        weights3
    ) + biases3, 
    0.5
  )
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
  )
  
  # Optimizer.
  # use smaller learning rate for weights to prevent loss become NaN
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf_valid_dataset, 
                        weights1
                    ) + biases1
                ),
                weights2
            ) + biases2
        ),
        weights3
    ) + biases3
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf_test_dataset, 
                        weights1
                    ) + biases1
                ),
                weights2
            ) + biases2
        ),
        weights3
    ) + biases3
  )

In [83]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 12.987659
Minibatch accuracy: 9.4%
Validation accuracy: 16.1%
Minibatch loss at step 500: 1.350211
Minibatch accuracy: 46.1%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 1.117716
Minibatch accuracy: 51.6%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 0.963965
Minibatch accuracy: 61.7%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 1.129193
Minibatch accuracy: 55.5%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 1.262647
Minibatch accuracy: 44.5%
Validation accuracy: 87.0%
Minibatch loss at step 3000: 1.230572
Minibatch accuracy: 50.0%
Validation accuracy: 87.5%
Minibatch loss at step 3500: 1.188814
Minibatch accuracy: 52.3%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.970662
Minibatch accuracy: 60.9%
Validation accuracy: 88.1%
Minibatch loss at step 4500: 1.114918
Minibatch accuracy: 50.0%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 1.092045
Minibatch accuracy: 53.1%
Validation accurac

In [88]:
'''
3 layers neural network with dropouts between hidden layers
'''
batch_size = 128
hidden_layer1_size = 2048
hidden_layer2_size = 512
beta = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([hidden_layer1_size]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([hidden_layer2_size]))

  weights3 = tf.Variable(
    tf.truncated_normal([hidden_layer2_size, num_labels], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(
    tf.nn.relu(
        tf.nn.dropout(
            tf.matmul(
                tf.nn.relu(
                    tf.nn.dropout(
                        tf.matmul(
                            tf_train_dataset, 
                            weights1
                        ) + biases1,
                        0.5
                    )
                ), 
                weights2
            ) + biases2,
            0.5
        )
    ),
    weights3
  ) + biases3

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    #+ beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  )
  
  # Optimizer.
  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.1, global_step, 1000, 0.65, staircase=True)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf_valid_dataset, 
                        weights1
                    ) + biases1
                ),
                weights2
            ) + biases2
        ),
        weights3
    ) + biases3
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf_test_dataset, 
                        weights1
                    ) + biases1
                ),
                weights2
            ) + biases2
        ),
        weights3
    ) + biases3
  )

In [89]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.545281
Minibatch accuracy: 7.8%
Validation accuracy: 21.4%
Minibatch loss at step 500: 0.723505
Minibatch accuracy: 78.9%
Validation accuracy: 82.5%
Minibatch loss at step 1000: 0.658516
Minibatch accuracy: 78.9%
Validation accuracy: 83.9%
Minibatch loss at step 1500: 0.467256
Minibatch accuracy: 83.6%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 0.718237
Minibatch accuracy: 79.7%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 0.691731
Minibatch accuracy: 75.8%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 0.706408
Minibatch accuracy: 80.5%
Validation accuracy: 85.6%
Minibatch loss at step 3500: 0.716278
Minibatch accuracy: 78.1%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 0.385437
Minibatch accuracy: 90.6%
Validation accuracy: 86.1%
Minibatch loss at step 4500: 0.593892
Minibatch accuracy: 81.2%
Validation accuracy: 86.2%
Minibatch loss at step 5000: 0.503053
Minibatch accuracy: 83.6%
Validation accurac

In [92]:
'''
4 layers neural network with dropouts
'''
batch_size = 128
hidden_layer1_size = 2048
hidden_layer2_size = 512
hidden_layer3_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # use smaller stddev for weights to prevent loss become NaN
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer1_size], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([hidden_layer1_size]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([hidden_layer2_size]))

  weights3 = tf.Variable(
    tf.truncated_normal([hidden_layer2_size, hidden_layer3_size], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([hidden_layer3_size]))

  weights4 = tf.Variable(
    tf.truncated_normal([hidden_layer3_size, num_labels], stddev=0.1))
  biases4 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.dropout(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf.nn.relu(
                            tf.matmul(
                                tf_train_dataset, 
                                weights1
                            ) + biases1
                        ), 
                        weights2
                    ) + biases2
                ),
                weights3
            ) + biases3
        ), 
        weights4
    ) + biases4, 
    0.5
  )

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
  )
  
  # Optimizer.
  # use smaller learning rate for weights to prevent loss become NaN
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf.nn.relu(
                            tf.matmul(
                                tf_valid_dataset, 
                                weights1
                            ) + biases1
                        ),
                        weights2
                    ) + biases2
                ),
                weights3
            ) + biases3
        ), 
        weights4
    ) + biases4
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf.nn.relu(
                            tf.matmul(
                                tf_test_dataset, 
                                weights1
                            ) + biases1
                        ),
                        weights2
                    ) + biases2
                ),
                weights3
            ) + biases3
        ), 
        weights4
    ) + biases4
  )

In [93]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.913697
Minibatch accuracy: 6.2%
Validation accuracy: 10.9%
Minibatch loss at step 500: 1.118322
Minibatch accuracy: 58.6%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 1.161325
Minibatch accuracy: 57.8%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 1.079207
Minibatch accuracy: 54.7%
Validation accuracy: 85.0%
Minibatch loss at step 2000: 1.322836
Minibatch accuracy: 46.9%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 1.040613
Minibatch accuracy: 57.0%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 1.043447
Minibatch accuracy: 58.6%
Validation accuracy: 86.1%
Minibatch loss at step 3500: 1.218679
Minibatch accuracy: 49.2%
Validation accuracy: 86.5%
Minibatch loss at step 4000: 1.078356
Minibatch accuracy: 54.7%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 1.199750
Minibatch accuracy: 46.9%
Validation accuracy: 87.1%
Minibatch loss at step 5000: 1.078882
Minibatch accuracy: 55.5%
Validation accuracy